# Cell correction

This notebook is for cells labels by deleting them, merge 2 neighboors cells or eventually extends them

# 0. Import 

In [1]:
from correct_cells import * 

In [2]:
import os
import copy
from pathlib import Path
import pandas as pd

import numpy as np
import napari
from napari.utils.notebook_display import nbscreenshot
from tqdm import tqdm
from rich.pretty import pprint

from dask.array.image import imread as imr


# 1. Load images and mask
You need a 2D projection of the movie and the all the segmentation labels detected throughout the movie

In [3]:
images_path = Path("/media/raphael/Genia_ordi/00_20240612_C3.45_ON_S2/MIP_C3.45_S2.tif")
labels_path = "/media/raphael/Genia_ordi/00_20240612_C3.45_ON_S2/labels" + "/*.tif"

In [4]:
labels = imr(labels_path)
labels = labels.compute()


In [5]:
viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
#viewer.open(sorted(images_path.glob("*.tif")), stack=True)
#viewer.open(sorted(labels_path.glob("*.tif")), stack=True)

In [6]:
viewer.open(images_path)

[<Image layer 'MIP_C3.45_S2' at 0x7f080819a6b0>]

In [7]:
images = viewer.layers[0].data
Label = viewer.add_labels(labels)


## 1.2 Create path for the cell_index.npy file 

In [8]:
cellIndexFolder = "/media/raphael/Genia_ordi/00_20240612_C3.45_ON_S2" + "/"
cellIndexName = "cell_index"
cellIndexFile = cellIndexFolder + cellIndexName

# 2. Correct cells  

## 2.1 get Number and index of cells present on all frames 

In [9]:
percentage,cells_res = get_cells_numbers_on_all_frames(labels)

previous number of cells :  170
actual max number of cells :  170
number of cells present from first to last frame :  17
percentage of cells present from first to last frame :  10.0 %


## 2.1.2 save cell index  

In [10]:
cells_res

[0, 33, 36, 38, 8, 42, 43, 44, 13, 14, 15, 16, 45, 18, 19, 20, 29, 31]

In [11]:
np.save(cellIndexFile,cells_res,allow_pickle=True, fix_imports=True)

## 2.2 Delete cells 

In dlt put a tuple corresponding to the labels of cells you want to delete and these cell will be successively deleted by "delete_cells"

In [21]:
# tuple corresponding to the numbers of cell which will be deleted 
dlt=[51] 
len_d = int(len(dlt))

In [22]:
for i in range(0,len_d):
    print("cell to be deleted ", dlt[i])
    labels = delete_cells(labels, dlt[i])
    labels = np.array(labels)
    viewer.layers[1].data = labels #refresh labels on napari

cell to be deleted  51
cell  51 has been deleted on all frames


## 2.3 Merge neighboor cells 

The same principle goes for merging cell, but this time you have to put the cells you wants to merge by pair 
like this nbr = (a,b,x,y)


a and b will be merge together, and it will be the same for x and y 

In [12]:
nbr=(30,151,30,162,22,150,49,65,49,107,48,122,48,135,48,155,48,164)
len_nbr = int(len(nbr))

In [ ]:
for i in range(0,len_nbr,2):
    print("parent cell ", nbr[i]," child cell ",nbr[i+1])
    labels = merge_neighboor_cells(nbr[i], nbr[i+1],labels)
    labels = np.array(labels)
    viewer.layers[1].data = labels #refresh labels on napari

parent cell  30  child cell  151
Cell  30 has been joined to cell  151
parent cell  30  child cell  162
Cell  30 has been joined to cell  162
parent cell  22  child cell  150
Cell  22 has been joined to cell  150
parent cell  49  child cell  65
Cell  49 has been joined to cell  65
parent cell  49  child cell  107
Cell  49 has been joined to cell  107
parent cell  48  child cell  122
Cell  48 has been joined to cell  122
parent cell  48  child cell  135


# 3 Save Corrected labels 

In [ ]:
savepath = "/media/raphael/Genia_ordi/00_20240612_C3.45_ON_S2/mask/"
save_zarr_images(labels,savepath,"c3.45_ON_01_w1SPI-488_s","2","mask")